In [29]:

from splinter import Browser
from bs4 import BeautifulSoup
import time
import pymongo
import requests
# from splinter.exceptions import ElementDoesNotExist
import pandas as pd
from flask_pymongo import PyMongo
from flask import Flask

In [30]:


def init_browser():

        executable_path = {"executable_path": "chromedriver.exe"}
        return Browser("chrome", **executable_path, headless=False)




In [3]:
# app = Flask(__name__)
# mongo = PyMongo(app, uri="mongodb://localhost:27017/tax_app")

# address = mongo.db.input.distinct('Query')
# address_list = []
# for x in address:
#     address_list.append(x)
# print(address)
# # address = 'Holston'
# browser = init_browser()
# url = 'https://property.spatialest.com/nc/durham/#/'
# browser.visit(url)
# html = browser.html
# soup = BeautifulSoup(html, 'html.parser')

# results = soup.find_all('input', class_='search')
# # print(results)

['Marne Ave']


In [4]:
# browser.find_by_id('primary_search').first.fill(address)

In [5]:
# browser.click_link_by_partial_href('#') 

In [6]:
# data = browser.url
# browser.visit(data)

# property_link1 = browser.html
# prop_link1 = BeautifulSoup(property_link1, 'html.parser')
# prop_link1
# prop_list = []
# addresses = prop_link1.findAll('div', class_="item-1")
# for address in addresses:
#     prop_list.append(address.text)
# prop_list

['1335 MARNE AVE',
 '1339 MARNE AVE',
 '1344 MARNE AVE',
 '1344 MARNE AVE, C',
 '1344 MARNE AVE, B',
 '1344 MARNE AVE, C',
 '1344 MARNE AVE, D',
 '1340 MARNE AVE',
 '1334 MARNE AVE',
 '1330 MARNE AVE',
 '1326 MARNE AVE',
 '1322 MARNE AVE',
 '1318 MARNE AVE',
 '1314 MARNE AVE',
 '1310 MARNE AVE',
 '1306 MARNE AVE',
 '1302 MARNE AVE',
 '1242 MARNE AVE',
 '1240 MARNE AVE',
 '1236 MARNE AVE',
 '1232 MARNE AVE']

In [52]:
app = Flask(__name__)
mongo = PyMongo(app, uri="mongodb://localhost:27017/tax_app")

address = mongo.db.input.distinct('Query')
address_list = []
for x in address:
    address_list.append(x)
print(address)


['Ripley St']


In [53]:
# address = 'Holston'
browser = init_browser()
url = 'https://property.spatialest.com/nc/durham/#/'
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

results = soup.find_all('input', class_='search')
browser.find_by_id('primary_search').first.fill(address)
time.sleep(3)
browser.click_link_by_partial_href('#')
time.sleep(3)
data = browser.url
browser.visit(data)

property_link1 = browser.html
prop_link1 = BeautifulSoup(property_link1, 'html.parser')
prop_link1
prop_list = []
addresses = prop_link1.findAll('div', class_="item-1")
for address in addresses:
    prop_list.append(address.text)
time.sleep(2)
next_list =  browser.find_by_text('Next')
time.sleep(2)

In [54]:
while len(next_list)>0:
    try:
        next_list =  browser.find_by_text('Next')
        browser.click_link_by_partial_text('Next') 
        time.sleep(3)
        data1 = browser.url
        browser.visit(data1)

        property_link1 = browser.html
        prop_link1 = BeautifulSoup(property_link1, 'html.parser')
        prop_link1

        addresses1 = prop_link1.findAll('div', class_="item-1")
        for address1 in addresses1:
            prop_list.append(address1.text)
        for next1 in next_list:
            next_list.remove(next1)
    except:
        prop_list

In [55]:
browser.click_link_by_partial_href('spatialest')
time.sleep(3)
prop_mailing1 = []    
address = []
price = []
date = []
while len(prop_list)>0:
    try:
        for prop in prop_list:
            browser.find_by_id('primary_search').first.fill(prop)
            time.sleep(3)
            browser.click_link_by_partial_href('#') 
            time.sleep(5)
            data = browser.url
            browser.visit(data)
            time.sleep(3)
            property_link3 = browser.html
            prop_link = BeautifulSoup(property_link3, 'html.parser')
            prop_address = prop_link.find('div', class_='location text-highlight')
            prop_address1 = prop_address.find('span', class_='value')
            prop_address
            if prop_address1.text == prop:
                
                prop_mailing = prop_link.find('div', class_='mailing')
                prop_mailing1.append(prop)
                address.append(prop)
                prop_table = prop_link.findAll('table')[3]
                price_table = prop_table.findAll('td')[1]
                date_table = prop_table.findAll('td')[0]
                for x in price_table:
                    price.append(x)
                for y in date_table:
                    date.append(y)
                browser.click_link_by_partial_href('spatialest')
                time.sleep(3)
                prop_list.remove(prop)
                print(len(prop_list))
            else:
                browser.click_link_by_partial_href('spatialest')
                time.sleep(3)
                prop_list.remove(prop)
                print(len(prop_list))
    except:

        prop_list.remove(prop)
        print(len(prop_list))
        browser.click_link_by_partial_href('spatialest')
        time.sleep(2)

2
1
0


In [56]:
df1 = pd.DataFrame(columns=['Address', 'Price', 'Date'])
df1['Address']=address
df1['Price']=price
df1['Date']=date
table = df1.to_html(classes="table")
table.replace('\n', '')
df1.to_html('table.html')
df = {
'Address': address, 
'Price': price, 
'Date': date,
'Table': table,
}

In [11]:
# df.to_csv('Barringer3.csv', encoding='utf-8', index=False)

In [27]:
# df1 = pd.read_csv('Barringer.csv')

In [57]:
mongo.db.collection.update({}, df, upsert=True)


c:\users\user\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  """Entry point for launching an IPython kernel.


{'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}

In [68]:
df1.loc[df1['Date'] == '2016 Q1', 'Date'] = 'No Recent Sales'
df1

,Address,Price,Date
0,2003 WARD ST,"338,000",No Recent Sales
1,808 BRYE ST,"81,000",No Recent Sales
2,1315 RIPLEY ST,"255,000",06/16/2006


In [75]:


date1 = ['No Recent Sales' if x=='2016 Q1' else x for x in date]

date1

['No Recent Sales', 'No Recent Sales', '06/16/2006']

In [59]:
data = browser.url
browser.visit(data)
time.sleep(3)
property_link3 = browser.html
prop_link = BeautifulSoup(property_link3, 'html.parser')
prop_address = prop_link.find('div', class_='location text-highlight')
prop_address1 = prop_address.find('span', class_='value')
prop_address1

AttributeError: 'NoneType' object has no attribute 'find'